# Import necessary libraries


In [2]:
import pandas as pd
import numpy as np
import lime

# import lime.lime_tabular
import matplotlib.pyplot as plt
import seaborn as sns

import shap

# Read the data


In [3]:
freezer_data = pd.read_csv("./Data Storn - Final round data sets/freezer_data.csv")
outlets_data = pd.read_csv("./Data Storn - Final round data sets/outlets_data.csv")
product_data = pd.read_csv("./Data Storn - Final round data sets/product_data.csv")
sales_data = pd.read_csv("./Data Storn - Final round data sets/sales_data.csv")
week_data = pd.read_csv("./Data Storn - Final round data sets/week_data.csv")

In [4]:
freezer_data.head()

,Freezer Name,Model Number,Volume Capacity (Liters),Power and maitainance Cost (LKR) per 100 hours
0,ChillMaster,M001,55,2500
1,FreezeZone,M002,75,2800
2,CoolTech,M003,120,3200
3,mini CoolTech,M004,30,1800
4,IceBlast Pro,M005,150,3500


## Freezer data


In [5]:
freezer_data.shape

(10, 4)

In [11]:
freezer_data["Model Number"].nunique()

10

## Outlet data


In [6]:
outlets_data.head()

,Outlet_ID,area(sqft)
0,ID7203,5660
1,ID1878,4138
2,ID1740,5546
3,ID4366,5134
4,ID8358,4100


In [7]:
outlets_data.shape

(988, 2)

In [8]:
outlets_data["Outlet_ID"].nunique()

951

In [9]:
outlets_data.drop_duplicates(subset=["Outlet_ID"], inplace=True)

In [10]:
outlets_data.shape

(951, 2)

## Product data


In [12]:
product_data.head()

,pid,volume,product_name,price,cost,stock
0,IP3,0.2,Strawberry Tiny Cup,110,98.45,225000
1,IP2,0.2,Chocolate Petite Bar,90,79.2,75000
2,IP1,0.2,Vanilla Mini Cone,100,89,120000
3,IP5,0.2,Butter Pecan Small Stick,100,88.5,150000
4,IP4,0.2,Mint Bite-size Pop,100,89.5,200000


In [13]:
product_data["pid"].nunique()

10

In [14]:
product_data.shape

(10, 6)

## Sales data


In [15]:
sales_data.head()

,Row Labels,Sum of total vlume,Sum of total sales,rank
0,IP3,1036253.2,113987852,1
1,IP2,1030783.2,92770488,2
2,IP1,1023574.2,102357420,3
3,IP5,1022602.4,102260240,4
4,IP4,1017999.0,101799900,5


In [16]:
sales_data.shape

(10, 4)

In [17]:
sales_data.head(10)

,Row Labels,Sum of total vlume,Sum of total sales,rank
0,IP3,1036253.2,113987852,1
1,IP2,1030783.2,92770488,2
2,IP1,1023574.2,102357420,3
3,IP5,1022602.4,102260240,4
4,IP4,1017999.0,101799900,5
5,BP2,922304.0,1014534400,6
6,BP1,920582.0,1104698400,7
7,BP5,912377.0,1003614700,8
8,BP3,909823.0,1046296450,9
9,BP4,903481.0,903481000,10


In [48]:
sales_data.rename(columns={"Row Labels": "pid"}, inplace=True)

## Week data


In [18]:
week_data.head()

,Week,Start Date,End Date
0,Week 1,1/2/2023,1/8/2023
1,Week 2,1/9/2023,1/15/2023
2,Week 3,1/16/2023,1/22/2023
3,Week 4,1/23/2023,1/29/2023
4,Week 5,1/30/2023,2/5/2023


# EDA


## NOTE:

-   We don't need week data
-   Freezer data contains 10 unique freezers
-   Outlet data contains 951 unique outlets with area
-   Product data contains 10 unique products with product_name, price cost, & stock
-   Sales data contains the total volume sold, and sum of total sales for each product in all the outlets


In [32]:
final_segmenation = pd.read_csv("final_segmentation.csv")

In [33]:
final_segmenation.head()

,cluster,min_fridge_id,min_fridge_cost,mean_weekly_volume_sold,mean_weekly_earning,nfridges,tot_freezer_capacity,ROI,item_sales_ratio
0,0,M005,35000.0,1469.319955,1.187665e+06,10.0,1500.0,33.933294,0.979547
1,1,M005,10500.0,436.892391,4.849972e+05,3.0,450.0,46.190210,0.970872
2,2,M010,13500.0,598.912539,3.760313e+05,3.0,600.0,27.854170,0.998188
3,3,M005,52500.0,2207.246190,1.742291e+06,15.0,2250.0,33.186500,0.980998


In [34]:
sandil2_df = pd.read_csv(
    "../DataStorm-4.0-BrainLeftException-SemiFinals/encoded_outlets.csv"
)

In [35]:
sandil2_df.head()

,Outlet_ID,0,1,2,cluster
0,ID7203,0.0,0.0,0.836982,2
1,ID1878,0.0,0.0,0.838970,2
2,ID1740,0.0,0.0,0.882608,2
3,ID4366,0.0,0.0,0.960229,2
4,ID8358,0.0,0.0,0.923287,2


In [38]:
shop_cnt = sandil2_df.groupby(["cluster"]).count().reset_index()

In [39]:
shop_cnt.head()

,cluster,Outlet_ID,0,1,2
0,0,299,299,299,299
1,1,230,230,230,230
2,2,424,424,424,424
3,3,35,35,35,35


In [40]:
shop_cnt = shop_cnt[["cluster", "Outlet_ID"]]

In [41]:
shop_cnt.rename(columns={"Outlet_ID": "shop_cnt"}, inplace=True)

In [44]:
final_segmenation = final_segmenation.merge(shop_cnt, on="cluster", how="left")

In [45]:
final_segmenation["total_volume_req"] = (
    final_segmenation["shop_cnt"] * final_segmenation["mean_weekly_volume_sold"]
)

In [46]:
final_segmenation.head()

,cluster,min_fridge_id,min_fridge_cost,mean_weekly_volume_sold,mean_weekly_earning,nfridges,tot_freezer_capacity,ROI,item_sales_ratio,shop_cnt,total_volume_req
0,0,M005,35000.0,1469.319955,1.187665e+06,10.0,1500.0,33.933294,0.979547,299,439326.666667
1,1,M005,10500.0,436.892391,4.849972e+05,3.0,450.0,46.190210,0.970872,230,100485.250000
2,2,M010,13500.0,598.912539,3.760313e+05,3.0,600.0,27.854170,0.998188,424,253938.916667
3,3,M005,52500.0,2207.246190,1.742291e+06,15.0,2250.0,33.186500,0.980998,35,77253.616667


In [50]:
product_data = product_data.merge(sales_data, on="pid", how="left")

In [51]:
product_data.head()

,pid,volume,product_name,price,cost,stock,Sum of total vlume,Sum of total sales,rank
0,IP3,0.2,Strawberry Tiny Cup,110,98.45,225000,1036253.2,113987852,1
1,IP2,0.2,Chocolate Petite Bar,90,79.2,75000,1030783.2,92770488,2
2,IP1,0.2,Vanilla Mini Cone,100,89,120000,1023574.2,102357420,3
3,IP5,0.2,Butter Pecan Small Stick,100,88.5,150000,1022602.4,102260240,4
4,IP4,0.2,Mint Bite-size Pop,100,89.5,200000,1017999.0,101799900,5


In [53]:
product_data["available_volume"] = product_data["stock"] * product_data["volume"]
product_data.head()

,pid,volume,product_name,price,cost,stock,Sum of total vlume,Sum of total sales,rank,available_volume
0,IP3,0.2,Strawberry Tiny Cup,110,98.45,225000,1036253.2,113987852,1,45000.0
1,IP2,0.2,Chocolate Petite Bar,90,79.2,75000,1030783.2,92770488,2,15000.0
2,IP1,0.2,Vanilla Mini Cone,100,89,120000,1023574.2,102357420,3,24000.0
3,IP5,0.2,Butter Pecan Small Stick,100,88.5,150000,1022602.4,102260240,4,30000.0
4,IP4,0.2,Mint Bite-size Pop,100,89.5,200000,1017999.0,101799900,5,40000.0


In [56]:
product_data.head()

,pid,volume,product_name,price,cost,stock,Sum of total vlume,Sum of total sales,rank,available_volume
0,IP3,0.2,Strawberry Tiny Cup,110,98.45,225000,1036253.2,113987852,1,45000.0
1,IP2,0.2,Chocolate Petite Bar,90,79.2,75000,1030783.2,92770488,2,15000.0
2,IP1,0.2,Vanilla Mini Cone,100,89,120000,1023574.2,102357420,3,24000.0
3,IP5,0.2,Butter Pecan Small Stick,100,88.5,150000,1022602.4,102260240,4,30000.0
4,IP4,0.2,Mint Bite-size Pop,100,89.5,200000,1017999.0,101799900,5,40000.0


In [62]:
# convert cost column to float
product_data["cost"] = product_data[" cost "].str.replace(",", "").astype(float)

In [63]:
product_data.columns.tolist()

['pid',
 'volume',
 'product_name',
 'price',
 ' cost ',
 'stock',
 'Sum of total vlume',
 'Sum of total sales',
 'rank',
 'available_volume',
 'cost']

In [64]:
product_data["profit_earned_per_prod"] = product_data["price"] - product_data["cost"]
product_data.head()

,pid,volume,product_name,price,cost,stock,Sum of total vlume,Sum of total sales,rank,available_volume,cost,profit_earned_per_prod
0,IP3,0.2,Strawberry Tiny Cup,110,98.45,225000,1036253.2,113987852,1,45000.0,98.45,11.55
1,IP2,0.2,Chocolate Petite Bar,90,79.2,75000,1030783.2,92770488,2,15000.0,79.20,10.80
2,IP1,0.2,Vanilla Mini Cone,100,89,120000,1023574.2,102357420,3,24000.0,89.00,11.00
3,IP5,0.2,Butter Pecan Small Stick,100,88.5,150000,1022602.4,102260240,4,30000.0,88.50,11.50
4,IP4,0.2,Mint Bite-size Pop,100,89.5,200000,1017999.0,101799900,5,40000.0,89.50,10.50


In [75]:
product_data.head(10)

,pid,volume,product_name,price,cost,stock,Sum of total vlume,Sum of total sales,rank,available_volume,cost,profit_earned_per_prod
0,IP3,0.2,Strawberry Tiny Cup,110,98.45,225000,1036253.2,113987852,1,45000.0,98.45,11.55
1,IP2,0.2,Chocolate Petite Bar,90,79.2,75000,1030783.2,92770488,2,15000.0,79.20,10.80
2,IP1,0.2,Vanilla Mini Cone,100,89,120000,1023574.2,102357420,3,24000.0,89.00,11.00
3,IP5,0.2,Butter Pecan Small Stick,100,88.5,150000,1022602.4,102260240,4,30000.0,88.50,11.50
4,IP4,0.2,Mint Bite-size Pop,100,89.5,200000,1017999.0,101799900,5,40000.0,89.50,10.50
5,BP2,1.0,Moose Tracks Jumbo Cup,1100,924,8000,922304.0,1014534400,6,8000.0,924.00,176.00
6,BP1,1.0,Chocolate Fudge Big Scoop,1200,"1,020.00",10000,920582.0,1104698400,7,10000.0,1020.00,180.00
7,BP5,1.0,Cookies and Cream Mega Tub,1100,918.5,4000,912377.0,1003614700,8,4000.0,918.50,181.50
8,BP3,1.0,Mint Chocolate Chip Family Pack,1150,966,9000,909823.0,1046296450,9,9000.0,966.00,184.00
9,BP4,1.0,Rocky Road Mega Tub,1000,820,5000,903481.0,903481000,10,5000.0,820.00,180.00


In [66]:
volume_of_each_prod = product_data["volume"].values.tolist()

In [67]:
volume_of_each_prod

[0.2, 0.2, 0.2, 0.2, 0.2, 1.0, 1.0, 1.0, 1.0, 1.0]

In [69]:
profit_earned_per_prod = product_data["profit_earned_per_prod"].values.tolist()
# profit_earned_per_prod
# round the values to 2 decimal places
profit_earned_per_prod = [round(num, 2) for num in profit_earned_per_prod]
profit_earned_per_prod

[11.55, 10.8, 11.0, 11.5, 10.5, 176.0, 180.0, 181.5, 184.0, 180.0]

In [70]:
total_volume_available_for_each_prod = product_data["available_volume"].values.tolist()
total_volume_available_for_each_prod

[45000.0,
 15000.0,
 24000.0,
 30000.0,
 40000.0,
 8000.0,
 10000.0,
 4000.0,
 9000.0,
 5000.0]

In [54]:
clustered_df = final_segmenation[
    [
        "cluster",
        "min_fridge_id",
        "mean_weekly_volume_sold",
        "tot_freezer_capacity",
        "shop_cnt",
    ]
]

clustered_df.head()

,cluster,min_fridge_id,mean_weekly_volume_sold,tot_freezer_capacity,shop_cnt
0,0,M005,1469.319955,1500.0,299
1,1,M005,436.892391,450.0,230
2,2,M010,598.912539,600.0,424
3,3,M005,2207.246190,2250.0,35


In [73]:
clustered_df["capacity_available_in_freezer_for_each_cluster"] = (
    clustered_df["tot_freezer_capacity"] * clustered_df["shop_cnt"]
)

clustered_df.head()


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,cluster,min_fridge_id,mean_weekly_volume_sold,tot_freezer_capacity,shop_cnt,total_volume_to_sell,capacity_available_in_freezer_for_each_cluster
0,0,M005,1469.319955,1500.0,299,439326.666667,448500.0
1,1,M005,436.892391,450.0,230,100485.250000,103500.0
2,2,M010,598.912539,600.0,424,253938.916667,254400.0
3,3,M005,2207.246190,2250.0,35,77253.616667,78750.0


In [77]:
clustered_df["daily_mean_volume_sold"] = clustered_df["mean_weekly_volume_sold"] / 7
clustered_df.head()


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,cluster,min_fridge_id,mean_weekly_volume_sold,tot_freezer_capacity,shop_cnt,total_volume_to_sell,capacity_available_in_freezer_for_each_cluster,daily_mean_volume_sold
0,0,M005,1469.319955,1500.0,299,439326.666667,448500.0,209.902851
1,1,M005,436.892391,450.0,230,100485.250000,103500.0,62.413199
2,2,M010,598.912539,600.0,424,253938.916667,254400.0,85.558934
3,3,M005,2207.246190,2250.0,35,77253.616667,78750.0,315.320884


In [74]:
freezer_vol = clustered_df[
    "capacity_available_in_freezer_for_each_cluster"
].values.tolist()
freezer_vol = [round(num, 2) * 0.3 for num in freezer_vol]
freezer_vol

[134550.0, 31050.0, 76320.0, 23625.0]

In [76]:
freezer_data.head(10)

,Freezer Name,Model Number,Volume Capacity (Liters),Power and maitainance Cost (LKR) per 100 hours
0,ChillMaster,M001,55,2500
1,FreezeZone,M002,75,2800
2,CoolTech,M003,120,3200
3,mini CoolTech,M004,30,1800
4,IceBlast Pro,M005,150,3500
5,IceBlast,M006,65,2700
6,FreezeTech,M007,100,3000
7,FreezeTech 3.0,M008,25,1600
8,ChillTech,M009,80,2800
9,IceTech,M010,200,4500


In [78]:
freezer_volume_dict = dict(
    zip(freezer_data["Model Number"], freezer_data["Volume Capacity (Liters)"])
)
freezer_cost_dict = dict(
    zip(
        freezer_data["Model Number"],
        freezer_data["Power and maitainance Cost (LKR) per 100 hours"],
    )
)

In [80]:
outlets_data_labeled_ = clustered_df.copy()
for f_id, f_cap in freezer_volume_dict.items():
    outlets_data_labeled_[f"{f_id}s_needed"] = np.ceil(
        outlets_data_labeled_["daily_mean_volume_sold"] / f_cap
    )
    outlets_data_labeled_[f"{f_id}s_cost"] = (
        outlets_data_labeled_[f"{f_id}s_needed"] * freezer_cost_dict[f_id]
    )

outlets_data_labeled_

,cluster,min_fridge_id,mean_weekly_volume_sold,tot_freezer_capacity,shop_cnt,total_volume_to_sell,capacity_available_in_freezer_for_each_cluster,daily_mean_volume_sold,M001s_needed,M001s_cost,...,M006s_needed,M006s_cost,M007s_needed,M007s_cost,M008s_needed,M008s_cost,M009s_needed,M009s_cost,M010s_needed,M010s_cost
0,0,M005,1469.319955,1500.0,299,439326.666667,448500.0,209.902851,4.0,10000.0,...,4.0,10800.0,3.0,9000.0,9.0,14400.0,3.0,8400.0,2.0,9000.0
1,1,M005,436.892391,450.0,230,100485.250000,103500.0,62.413199,2.0,5000.0,...,1.0,2700.0,1.0,3000.0,3.0,4800.0,1.0,2800.0,1.0,4500.0
2,2,M010,598.912539,600.0,424,253938.916667,254400.0,85.558934,2.0,5000.0,...,2.0,5400.0,1.0,3000.0,4.0,6400.0,2.0,5600.0,1.0,4500.0
3,3,M005,2207.246190,2250.0,35,77253.616667,78750.0,315.320884,6.0,15000.0,...,5.0,13500.0,4.0,12000.0,13.0,20800.0,4.0,11200.0,2.0,9000.0


In [81]:
def get_min_cost(row):
    freezer_ids = list(freezer_volume_dict.keys())
    freezer_ids.sort()
    min = np.inf
    for f_id in freezer_ids:
        if row[f"{f_id}s_cost"] < min:
            min = row[f"{f_id}s_cost"]

    return min


def get_min_fid(row):
    freezer_ids = list(freezer_volume_dict.keys())
    freezer_ids.sort()
    min = np.inf
    min_f_id = ""
    for f_id in freezer_ids:
        if row[f"{f_id}s_cost"] < min:
            min = row[f"{f_id}s_cost"]
            min_f_id = f_id

    return min_f_id

In [82]:
outlets_data_labeled_["min_fridge_cost"] = outlets_data_labeled_.apply(
    get_min_cost, axis=1
)
outlets_data_labeled_["min_fridge_id"] = outlets_data_labeled_.apply(
    get_min_fid, axis=1
)

outlets_data_labeled_

,cluster,min_fridge_id,mean_weekly_volume_sold,tot_freezer_capacity,shop_cnt,total_volume_to_sell,capacity_available_in_freezer_for_each_cluster,daily_mean_volume_sold,M001s_needed,M001s_cost,...,M006s_cost,M007s_needed,M007s_cost,M008s_needed,M008s_cost,M009s_needed,M009s_cost,M010s_needed,M010s_cost,min_fridge_cost
0,0,M003,1469.319955,1500.0,299,439326.666667,448500.0,209.902851,4.0,10000.0,...,10800.0,3.0,9000.0,9.0,14400.0,3.0,8400.0,2.0,9000.0,6400.0
1,1,M006,436.892391,450.0,230,100485.250000,103500.0,62.413199,2.0,5000.0,...,2700.0,1.0,3000.0,3.0,4800.0,1.0,2800.0,1.0,4500.0,2700.0
2,2,M007,598.912539,600.0,424,253938.916667,254400.0,85.558934,2.0,5000.0,...,5400.0,1.0,3000.0,4.0,6400.0,2.0,5600.0,1.0,4500.0,3000.0
3,3,M010,2207.246190,2250.0,35,77253.616667,78750.0,315.320884,6.0,15000.0,...,13500.0,4.0,12000.0,13.0,20800.0,4.0,11200.0,2.0,9000.0,9000.0


In [83]:
final_cluster_df = outlets_data_labeled_[
    ["cluster", "min_fridge_id", "shop_cnt", "daily_mean_volume_sold"]
]
final_cluster_df.head()

,cluster,min_fridge_id,shop_cnt,daily_mean_volume_sold
0,0,M003,299,209.902851
1,1,M006,230,62.413199
2,2,M007,424,85.558934
3,3,M010,35,315.320884


In [85]:
shop_info = pd.read_csv(
    "../DataStorm-4.0-BrainLeftException-SemiFinals/with_outlets.csv"
)

In [86]:
shop_info.head()

,Outlet_ID,weekly_sale_for_1,weekly_sale_for_2,weekly_sale_for_3,weekly_sale_for_4,weekly_sale_for_5,weekly_sale_for_6,weekly_sale_for_7,weekly_sale_for_8,weekly_sale_for_9,...,Chocolate Fudge Big Scoop_earning,Cookies and Cream Mega Tub_earning,Mint Chocolate Chip Family Pack_earning,Moose Tracks Jumbo Cup_earning,Rocky Road Mega Tub_earning,mean_weekly_earning,mean_weekly_units_sold,mean_weekly_volume_sold,area(sqft),cluster
0,ID7203,3389.0,2402.0,2805.0,2583.0,2802.0,3227.0,2992.0,3138.0,2309.0,...,0.0,0.0,0.0,0.0,0.0,291076.666667,2895.500000,579.100000,5660,2
1,ID1878,2597.0,2048.0,2809.0,3724.0,3213.0,3313.0,2883.0,3501.0,3312.0,...,0.0,0.0,0.0,0.0,0.0,300528.333333,3003.333333,600.666667,4138,2
2,ID1740,3742.0,3208.0,3702.0,3060.0,3822.0,3190.0,2138.0,2812.0,2072.0,...,0.0,0.0,0.0,0.0,0.0,312992.500000,3132.250000,626.450000,5546,2
3,ID4366,3410.0,3072.0,3843.0,4036.0,2328.0,2454.0,3181.0,3025.0,3975.0,...,0.0,0.0,0.0,0.0,0.0,338344.166667,3358.666667,671.733333,5134,2
4,ID8358,2891.0,4185.0,3433.0,4552.0,1080.0,3598.0,3164.0,3497.0,3738.0,...,0.0,0.0,0.0,0.0,0.0,326558.333333,3281.500000,656.300000,4100,2


In [87]:
shop_info["daily_mean_volume_sold"] = shop_info["mean_weekly_volume_sold"] / 7

In [88]:
shop_info = shop_info.merge(
    final_cluster_df[["cluster", "min_fridge_id"]], on="cluster", how="left"
)

In [89]:
shop_info.head()

,Outlet_ID,weekly_sale_for_1,weekly_sale_for_2,weekly_sale_for_3,weekly_sale_for_4,weekly_sale_for_5,weekly_sale_for_6,weekly_sale_for_7,weekly_sale_for_8,weekly_sale_for_9,...,Mint Chocolate Chip Family Pack_earning,Moose Tracks Jumbo Cup_earning,Rocky Road Mega Tub_earning,mean_weekly_earning,mean_weekly_units_sold,mean_weekly_volume_sold,area(sqft),cluster,daily_mean_volume_sold,min_fridge_id
0,ID7203,3389.0,2402.0,2805.0,2583.0,2802.0,3227.0,2992.0,3138.0,2309.0,...,0.0,0.0,0.0,291076.666667,2895.500000,579.100000,5660,2,82.728571,M007
1,ID1878,2597.0,2048.0,2809.0,3724.0,3213.0,3313.0,2883.0,3501.0,3312.0,...,0.0,0.0,0.0,300528.333333,3003.333333,600.666667,4138,2,85.809524,M007
2,ID1740,3742.0,3208.0,3702.0,3060.0,3822.0,3190.0,2138.0,2812.0,2072.0,...,0.0,0.0,0.0,312992.500000,3132.250000,626.450000,5546,2,89.492857,M007
3,ID4366,3410.0,3072.0,3843.0,4036.0,2328.0,2454.0,3181.0,3025.0,3975.0,...,0.0,0.0,0.0,338344.166667,3358.666667,671.733333,5134,2,95.961905,M007
4,ID8358,2891.0,4185.0,3433.0,4552.0,1080.0,3598.0,3164.0,3497.0,3738.0,...,0.0,0.0,0.0,326558.333333,3281.500000,656.300000,4100,2,93.757143,M007


In [91]:
shop_info.rename(columns={"min_fridge_id": "Model Number"}, inplace=True)

In [94]:
freezer_data.head(10)

,Freezer Name,Model Number,Volume Capacity (Liters),Power and maitainance Cost (LKR) per 100 hours
0,ChillMaster,M001,55,2500
1,FreezeZone,M002,75,2800
2,CoolTech,M003,120,3200
3,mini CoolTech,M004,30,1800
4,IceBlast Pro,M005,150,3500
5,IceBlast,M006,65,2700
6,FreezeTech,M007,100,3000
7,FreezeTech 3.0,M008,25,1600
8,ChillTech,M009,80,2800
9,IceTech,M010,200,4500


In [95]:
shop_info = shop_info.merge(
    freezer_data[["Model Number", "Volume Capacity (Liters)"]],
    on="Model Number",
    how="left",
)

In [126]:
shop_info.drop_duplicates(subset=["Outlet_ID"], inplace=True)

In [141]:
shop_info.head()

,Outlet_ID,weekly_sale_for_1,weekly_sale_for_2,weekly_sale_for_3,weekly_sale_for_4,weekly_sale_for_5,weekly_sale_for_6,weekly_sale_for_7,weekly_sale_for_8,weekly_sale_for_9,...,Rocky Road Mega Tub_earning,mean_weekly_earning,mean_weekly_units_sold,mean_weekly_volume_sold,area(sqft),cluster,daily_mean_volume_sold,Model Number,Volume Capacity (Liters),total_freezers_needed
0,ID7203,3389.0,2402.0,2805.0,2583.0,2802.0,3227.0,2992.0,3138.0,2309.0,...,0.0,291076.666667,2895.500000,579.100000,5660,2,82.728571,M007,100,1.0
1,ID1878,2597.0,2048.0,2809.0,3724.0,3213.0,3313.0,2883.0,3501.0,3312.0,...,0.0,300528.333333,3003.333333,600.666667,4138,2,85.809524,M007,100,1.0
2,ID1740,3742.0,3208.0,3702.0,3060.0,3822.0,3190.0,2138.0,2812.0,2072.0,...,0.0,312992.500000,3132.250000,626.450000,5546,2,89.492857,M007,100,1.0
3,ID4366,3410.0,3072.0,3843.0,4036.0,2328.0,2454.0,3181.0,3025.0,3975.0,...,0.0,338344.166667,3358.666667,671.733333,5134,2,95.961905,M007,100,1.0
4,ID8358,2891.0,4185.0,3433.0,4552.0,1080.0,3598.0,3164.0,3497.0,3738.0,...,0.0,326558.333333,3281.500000,656.300000,4100,2,93.757143,M007,100,1.0


In [128]:
shop_info["total_freezers_needed"] = np.ceil(
    shop_info["daily_mean_volume_sold"] / shop_info["Volume Capacity (Liters)"]
)

In [129]:
shop_info = pd.DataFrame(shop_info)

In [130]:
num_of_freezers = (
    shop_info.groupby(["Model Number"])["total_freezers_needed"].sum().reset_index()
)

In [131]:
num_of_freezers.head()

,Model Number,total_freezers_needed
0,M003,586.0
1,M006,359.0
2,M007,481.0
3,M010,39.0


In [132]:
cluster_one_freezer = shop_info[shop_info["cluster"] == 1]

In [133]:
cluster_one_freezer["total_freezers_needed"].unique()

array([2., 1.])

In [134]:
cluster_two_freezer = shop_info[shop_info["cluster"] == 2]

cluster_two_freezer["total_freezers_needed"].unique()

array([1., 2.])

In [135]:
cluster_three_freezer = shop_info[shop_info["cluster"] == 3]

cluster_three_freezer["total_freezers_needed"].unique()

array([2., 3.])

In [136]:
cluster_four_freezer = shop_info[shop_info["cluster"] == 0]

cluster_four_freezer["total_freezers_needed"].unique()

array([2., 3.])

In [137]:
freezer_data.head()

,Freezer Name,Model Number,Volume Capacity (Liters),Power and maitainance Cost (LKR) per 100 hours
0,ChillMaster,M001,55,2500
1,FreezeZone,M002,75,2800
2,CoolTech,M003,120,3200
3,mini CoolTech,M004,30,1800
4,IceBlast Pro,M005,150,3500


In [138]:
num_of_freezers.head()

,Model Number,total_freezers_needed
0,M003,586.0
1,M006,359.0
2,M007,481.0
3,M010,39.0


In [139]:
final_cluster_df.head()

,cluster,min_fridge_id,daily_mean_volume_sold
0,0,M003,209.902851
1,1,M006,62.413199
2,2,M007,85.558934
3,3,M010,315.320884


In [142]:
final_cluster_df.rename(columns={"min_fridge_id": "Model Number"}, inplace=True)


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [143]:
final_cluster_df = final_cluster_df.merge(
    num_of_freezers, on="Model Number", how="left"
)

final_cluster_df.head()

,cluster,Model Number,daily_mean_volume_sold,total_freezers_needed
0,0,M003,209.902851,586.0
1,1,M006,62.413199,359.0
2,2,M007,85.558934,481.0
3,3,M010,315.320884,39.0


In [145]:
final_cluster_df = final_cluster_df.merge(freezer_data, on="Model Number", how="left")

In [146]:
final_cluster_df.head()

,cluster,Model Number,daily_mean_volume_sold,total_freezers_needed,Freezer Name,Volume Capacity (Liters),Power and maitainance Cost (LKR) per 100 hours
0,0,M003,209.902851,586.0,CoolTech,120,3200
1,1,M006,62.413199,359.0,IceBlast,65,2700
2,2,M007,85.558934,481.0,FreezeTech,100,3000
3,3,M010,315.320884,39.0,IceTech,200,4500


In [147]:
final_cluster_df.drop(['Power and maitainance Cost (LKR) per 100 hours'], axis=1, inplace=True)

In [148]:
final_cluster_df.head()

,cluster,Model Number,daily_mean_volume_sold,total_freezers_needed,Freezer Name,Volume Capacity (Liters)
0,0,M003,209.902851,586.0,CoolTech,120
1,1,M006,62.413199,359.0,IceBlast,65
2,2,M007,85.558934,481.0,FreezeTech,100
3,3,M010,315.320884,39.0,IceTech,200


In [149]:
# Have to calculate the total volume available in all shops

final_cluster_df["total_volume_available"] = (
    final_cluster_df["total_freezers_needed"]
    * final_cluster_df["Volume Capacity (Liters)"]
)

final_cluster_df.head()

,cluster,Model Number,daily_mean_volume_sold,total_freezers_needed,Freezer Name,Volume Capacity (Liters),total_volume_available
0,0,M003,209.902851,586.0,CoolTech,120,70320.0
1,1,M006,62.413199,359.0,IceBlast,65,23335.0
2,2,M007,85.558934,481.0,FreezeTech,100,48100.0
3,3,M010,315.320884,39.0,IceTech,200,7800.0


# Optimization Time

In [150]:
final_cluster_df.head()

,cluster,Model Number,daily_mean_volume_sold,total_freezers_needed,Freezer Name,Volume Capacity (Liters),total_volume_available
0,0,M003,209.902851,586.0,CoolTech,120,70320.0
1,1,M006,62.413199,359.0,IceBlast,65,23335.0
2,2,M007,85.558934,481.0,FreezeTech,100,48100.0
3,3,M010,315.320884,39.0,IceTech,200,7800.0


In [151]:
product_data.head(10)

,pid,volume,product_name,price,cost,stock,Sum of total vlume,Sum of total sales,rank,available_volume,cost,profit_earned_per_prod
0,IP3,0.2,Strawberry Tiny Cup,110,98.45,225000,1036253.2,113987852,1,45000.0,98.45,11.55
1,IP2,0.2,Chocolate Petite Bar,90,79.2,75000,1030783.2,92770488,2,15000.0,79.20,10.80
2,IP1,0.2,Vanilla Mini Cone,100,89,120000,1023574.2,102357420,3,24000.0,89.00,11.00
3,IP5,0.2,Butter Pecan Small Stick,100,88.5,150000,1022602.4,102260240,4,30000.0,88.50,11.50
4,IP4,0.2,Mint Bite-size Pop,100,89.5,200000,1017999.0,101799900,5,40000.0,89.50,10.50
5,BP2,1.0,Moose Tracks Jumbo Cup,1100,924,8000,922304.0,1014534400,6,8000.0,924.00,176.00
6,BP1,1.0,Chocolate Fudge Big Scoop,1200,"1,020.00",10000,920582.0,1104698400,7,10000.0,1020.00,180.00
7,BP5,1.0,Cookies and Cream Mega Tub,1100,918.5,4000,912377.0,1003614700,8,4000.0,918.50,181.50
8,BP3,1.0,Mint Chocolate Chip Family Pack,1150,966,9000,909823.0,1046296450,9,9000.0,966.00,184.00
9,BP4,1.0,Rocky Road Mega Tub,1000,820,5000,903481.0,903481000,10,5000.0,820.00,180.00


In [155]:
# Volume of each product
volume_of_each_product = dict(
    zip(product_data["pid"], product_data["volume"])
)

# profit of each product
profit_of_each_product = dict(
    zip(product_data["pid"], product_data["profit_earned_per_prod"])
)

# Available volume of each product
available_volume_of_each_product = dict(
    zip(product_data["pid"], product_data["available_volume"])
)

# Space available in clusters
# 30% of the total volume available in each cluster is considered as max the space available in each cluster
# for a single product
space_available_in_clusters = dict(
    zip(final_cluster_df["cluster"], final_cluster_df["total_volume_available"]*0.3)
)

In [156]:
final_cluster_df.head()

,cluster,Model Number,daily_mean_volume_sold,total_freezers_needed,Freezer Name,Volume Capacity (Liters),total_volume_available
0,0,M003,209.902851,586.0,CoolTech,120,70320.0
1,1,M006,62.413199,359.0,IceBlast,65,23335.0
2,2,M007,85.558934,481.0,FreezeTech,100,48100.0
3,3,M010,315.320884,39.0,IceTech,200,7800.0


In [157]:
final_cluster_df['Max_selling_item'] = ['IP2', 'BP1', 'IP1', 'IP5']
final_cluster_df.head()

,cluster,Model Number,daily_mean_volume_sold,total_freezers_needed,Freezer Name,Volume Capacity (Liters),total_volume_available,Max_selling_item
0,0,M003,209.902851,586.0,CoolTech,120,70320.0,IP2
1,1,M006,62.413199,359.0,IceBlast,65,23335.0,BP1
2,2,M007,85.558934,481.0,FreezeTech,100,48100.0,IP1
3,3,M010,315.320884,39.0,IceTech,200,7800.0,IP5


In [159]:
prod_vol_df = product_data[['pid', 'volume']]
prod_vol_df.head()

,pid,volume
0,IP3,0.2
1,IP2,0.2
2,IP1,0.2
3,IP5,0.2
4,IP4,0.2


In [160]:
prod_vol_df.rename(columns={'pid': 'Max_selling_item', 'volume': 'Max_selling_item_volume'}, inplace=True)
prod_vol_df.head()


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Max_selling_item,Max_selling_item_volume
0,IP3,0.2
1,IP2,0.2
2,IP1,0.2
3,IP5,0.2
4,IP4,0.2


In [161]:
final_cluster_df = final_cluster_df.merge(prod_vol_df, on='Max_selling_item', how='left')
final_cluster_df.head()

,cluster,Model Number,daily_mean_volume_sold,total_freezers_needed,Freezer Name,Volume Capacity (Liters),total_volume_available,Max_selling_item,Max_selling_item_volume
0,0,M003,209.902851,586.0,CoolTech,120,70320.0,IP2,0.2
1,1,M006,62.413199,359.0,IceBlast,65,23335.0,BP1,1.0
2,2,M007,85.558934,481.0,FreezeTech,100,48100.0,IP1,0.2
3,3,M010,315.320884,39.0,IceTech,200,7800.0,IP5,0.2


In [162]:
final_cluster_df['30%_of_total_volume_available'] = final_cluster_df['total_volume_available']*0.3
final_cluster_df.head()

,cluster,Model Number,daily_mean_volume_sold,total_freezers_needed,Freezer Name,Volume Capacity (Liters),total_volume_available,Max_selling_item,Max_selling_item_volume,30%_of_total_volume_available
0,0,M003,209.902851,586.0,CoolTech,120,70320.0,IP2,0.2,21096.0
1,1,M006,62.413199,359.0,IceBlast,65,23335.0,BP1,1.0,7000.5
2,2,M007,85.558934,481.0,FreezeTech,100,48100.0,IP1,0.2,14430.0
3,3,M010,315.320884,39.0,IceTech,200,7800.0,IP5,0.2,2340.0
